<a href="https://colab.research.google.com/github/YuansongFeng/MadMario/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-MVP tutorial for walking users through building a learning Mario. Guidelines for creating this notebook (feel free to add/edit):
1. Extensive explanation (link to AI cheatsheet where necessary) 
2. Only ask for core logics


reinforcement learning is taking over -- learn it

we put together a tutorial project to get you up to speed -- as well as a intro rl handbook for reference! Along the tutorials, YOU will code up the project


it'd be helpful if you have basic knowlegde of python to work through this notebook, and some knowledge about math/stats to fully understand the rl cheatsheet. if anywhere it's unclear, please leave comments in the notebook directly! we will gather all your helpful reviews 

lets get going! first thing first, let's see what you will be building. A mario that playes the game numerous time, improve itself and eventually pass a level. In codewise, this corresponds to . 



In [0]:
# pseudo code
for i in epriosdes:
  act
  rem
  learn

in rl terminology, this is saying: . environment and agent are two key concepts in rl at play. We refer the concepts in rl cheathsheet in multiple places. it'd be helpful to review it, before we start coding. 


data pre-processing is a key step in machine learing, whether it'd be trianing a image classifier or a text generator. in rl we also pre-process data -- data here are observarions of the environment. 

# Environment
In the below section, you will pre-process the environment by turning the perceived RGB images into gray-scale images. The advantage of doing this is that now the model can be significantly smaller because the input channels turn from 3 to 1. Due to a reduced number of model parameters to learn, the training will be faster. 

To visualize what your pre-processing logic will do, here are the environment feedback to Mario before and after the pre-processing:

**before pre-processing**

![picture](https://drive.google.com/uc?id=1c9-tUWFyk4u_vNNrkZo1Rg0e2FUcbF3N)
![picture](https://drive.google.com/uc?id=1s7UewXkmF4g_gZfD7vloH7n1Cr-D3YYX)
![picture](https://drive.google.com/uc?id=1mXDt8rFLKT9a-YvhGOgGZT4bq0T2y7iw)

**after pre-processing**

![picture](https://drive.google.com/uc?id=1ED9brgnbPmUZL43Bl_x2FDmXd-hsHBQt)
![picture](https://drive.google.com/uc?id=1PB1hHSPk6jIhSxVok2u2ntHjvE3zrk7W)
![picture](https://drive.google.com/uc?id=1CYm5q71f_OlY_mqvZADuMMjPmcMgbjVW)

To pre-process the environment, we use the idea of a *wrapper*. By wrapping the environment, we can specify a desired pre-processing step to the environment output, specifically, the observation.  

Example of applying an environment wrapper:
```
env = ResizeObservation(env, shape=84)
```
In this case, the environment observation output is resized to a dimension of 84 x 84. 




## Wrappers

**Instructions**

We want to apply 3 built-in wrappers to the given `env`, `GrayScaleObservation`, `ResizeObservation`, and `FrameStack`.  

https://github.com/openai/gym/tree/master/gym/wrappers


`FrameStack` is a wrapper that will allow us to squash consecutive frames of the environment into a single observation point to feed to our learning model. This way, we can differentiate between when Mario was landing or jumping based on his direction of movement in the previous several frames. 

We can start with the following arguments:
`GrayScaleObservation`: keep_dim=False 
`ResizeObservation`: shape=84 
`FrameStack`: num_stack=4 


In [0]:
from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation
import gym_super_mario_bros

# the original environment object 
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')

def apply_wrappers_to_env(env):
    # TODO wrap the given env with GrayScaleObservation, ResizeObservation and FrameStack and return result

    # TODO wrap the given env with GrayScaleObservation and return result
    env = None
    # TODO wrap the given env with ResizeObservation and return result
    env = None
    # TODO wrap the given env with FrameStack and return result
    env = None

    return env

## Custom Wrapper

We also would like you to get a taste of implementing an environment wrapper on your own, instead of calling off-the-shelf packages. 

Here is an idea:
As an effort of downsizing our model to make training faster, we can choose to skip every n-th frame. In other words, our wrapped environment will only output every n-th frame. Below is a skeleton of the class `SkipFrame`, inherited from `gym.Wrapper`.  Notice in the `__init__` function, the `_skip` field is overriden by the input parameter, default set at 4.


However, it is important to accumulate the reward during these skipped steps, because the reward is the most important factor in determining the success of the learning model, so while we can skip frame for dimension reduction purpose, it is crucial we keep adding those rewards to our total reward. 


**Instruction**

Implement the reward accumulation function, using your favorite for loop.

In [0]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = None
            if done:
                break

        return obs, total_reward, done, info
       


**Instruction**

After you finished the `SkipFrame` class, you can call it on your preprocessed `env`, along with other three build-in wrappers.

In [0]:
# wrapper.py
import gym
import numpy as np
from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = None
            if done:
                break

        return obs, total_reward, done, info

def apply_wrappers_to_env(env):
    # TODO wrap the given env with GrayScaleObservation and return result
    env = None
    # TODO wrap the given env with ResizeObservation and return result
    env = None
    # TODO wrap the given env with FrameStack and return result
    env = None
    # TODO wrap the given env with SkipEnv
    env = SkipFrame(env, skip = 4)
    return None

# Agent

Agent and environment are two core concepts in reinforcement learning. The agent continuously interacts with the environment, collects reward and learn to maximize its overall return in the long term. In our scenario, Mario is the agent and other game components (blocks, tubes, mushrooms, etc.) are the environment. 

The agent class, `DQNAgent`, captures Mario's behavior in the game environment. The agent should be able to 

- Make its decision about next action to take. This requires Mario to process the environment state and find the optimal action that yields the highest return value. Refer to Optimal Action in Cheatsheet. 

- Remember past experiences. Mario should be able to add the current experience to its memory. Later, it uses all the previous experiences to learn to act smarter. 

- Learn to improve action over time. The decision made by Mario should yield higher and higher return as the training proceeds. This requires Mario to update its decision process based on previous experiences. Refer to Q-learning in the RL Cheatsheet. 


In [0]:
class DQNAgent:
    def __init__(self, state_dim, action_dim, max_memory, double_q):
        pass

    def act(self, state):
        """Given a state, choose an epsilon-greedy action
        """
        pass

    def remember(self, experience):
        """Add the observation to memory
        """
        pass

    def learn(self):
        """Update online action value (Q) function with a batch of experiences
        """
        pass


## Initialize
Before implementing the core functions, we need to first declare some attributes(variables) the agent will need to regulate its behaviors, some examples include epsilon(random exploration rate), epsilon decay rate, future reward discount rate, etc. Please refer to cheat sheet for more details.

eps (real number):
> Random Exploration Prabability. Under some probability, agent will not follow the policy(perform the best action),instead, it will randomly choose an action to explore the state space. This is very important at the early stage of learning, because agent does not have a good policy in the begining, it needs to try different actions to see which actions leads to better rewards. Random exploration also helps agent to fall into the local optima. eps will gradually decrease as agent's policy becomes better and better. 

> Please initialize it to 1.0.



eps_decay (real number):

> Decay rate of eps. Agent rigorously explores space at the early stage, but gradually reduces its exploration rate to maintain action quality. In the later stage, agent already learns a fairly good policy, so we want it to follow its policy more frequently. Decrease eps by the factor of eps_decay each time the agent acts.

> Please initialize it to 0.99999975


gamma (real number):
> Future reward discount rate. gamma serves to make agent give higher weight on the short-term rewards over future reward.

>Please initialize it to 0.9

batch_size (integer):
> number of experiences used to update neu each time.

> Please initialize it to 32

state_dim (tuple):

> state is the observation of the current environment which includes locations of obstacles, opponents, etc. The agent chooses the best action based entirely on the state. state_dim is the dimension of the state, in mario example, it is 4 consecutive snapshots of the enviroment stacked together, and each snapshot is a 84*84 gray-scale picture, so state_dim = (4, 84,84).

action_dim (integer/tuple):
>  The number of all possible actions the agent can perform.

max_memory (integer):
> The agent needs to remember its past experience(what the current state looks like, what action it performs, what state the action leads to, and the reward collected after performing the action). The experiences are stored in a double-ended queue(deque). During the learning phase, agent samples batch_size past experiences and update its policy. max_memory defines the capacity of the deque. If max_memory is 100, agent would remember its last 100 experiences. deque is FIFO and would throw away oldest experiences when capacity is reached and new experience is collected.

Note: You can always try other combinations of parameters and test how agent would behave.

In [0]:
class DQNAgent:
  
    def __init__(self, state_dim, action_dim, max_memory):
       # state space dimension
      self.state_dim = state_dim
      # action space dimension
      self.action_dim = action_dim
      # replay buffer
      self.memory = deque(maxlen=max_memory)
      # current step, updated everytime the agent acts
      self.step = 0

      #TODO: Please initialize other variables as described above


      pass
    


## Predict Q value

### `agent.predict()`
Before we work on `agent.act()` and `agent.learn()`, we will implement an important helper method that gets used all the time: `agent.predict(state)`. Given an environment state, this function calculates the estimated Q values for all possible actions. In other words, this is the function *Q\*(s, a)* in our Cheatsheet. Note the Q value function here is the *optimal value action function* (Refer to Optimal Value Action Function in Cheatsheet). 

In `agent.act()`, we use `predict()` to give us a list of Q values for all possible actions. Then we can choose the action with the highest Q value, as we are following a greedy policy. In `agent.learn()`, we rely on the Q-learning algorithm outlined in the Cheatsheet, which heavily depends on the function *Q\*(s, a)*. 


### Online and Target Q functions

Looking closer at the Q-learning algorithm in cheatsheet, we use *Q\*(s, a)* to define both the *TD target* (r + \gamma max_a Q(s', a')) and current state-action value (Q(s, a)). While mathematically Q(s', a') and Q(s, a) are all the same Q function, in practice we use a separate function to approximate each. This is to prevent the divergence problem during optimization. We use Q_online to represent the Q(s, a) function, and Q_target to represent the Q(s', a') function. Q_online is used to make actual action decision by the agent in `agent.act()`, and is being optimized upon in `agent.learn()`. Q_target is used to determine the optimization target of Q_online. 


### Q function:  convolution neural network
We use a neural network to approximates each Q function. The input to the Q function are game state, which is represented as a stack of frames. Outputs from Q function are the Q values corresponding to each state-action pair. Because the inputs are in image (stack of images) format, we use a *convolution neural network*. 

Ideally, the Q function outputs a single Q value for a given state-action pair. For a given state, if we have 5 possible actions to take, this would require running through Q function/neural network 5 times. This is costly. Instead, we let the Q function directly output 5 values corresponding to all possible actions, in a single run. 

### Calling neural network
We have implemented an off-the-shelf *convolution neural network* for you. To use the network, first initialize the network with state dimension and output dimension. State dimension is the dimension of environment observation, and output dimension is number of possible actions. Example:
```
self.neural_net = ConvNet(input_dim=state_dim, output_dim=action_dim)
```

After initialization, call the neural network with 
```
q_values = self.neural_net(state)
```


In [0]:
class DQNAgent:
    def __init__(self, ...):
      # online action value function, Q(s, a)
      self.online_q = ConvNet(input_dim=state_dim, output_dim=action_dim).to(self.device)
      # target action value function, Q'(s, a)
      self.target_q = ConvNet(input_dim=state_dim, output_dim=action_dim).to(self.device)

    def predict(self, state, model):
        """Given a state, predict Q values of all possible actions using specified model (either online or target)
        Input:
          state
           dimension of (batch_size * state_dim)
          model
           either 'online' or 'target'
        Output
          pred_q_values (torch.tensor)
            dimension of (batch_size * action_dim), predicted Q values for all possible actions given the state
        """
        # LazyFrame -> np array -> torch tensor
        state_float = torch.FloatTensor(np.array(state)).to(self.device)
        # normalize
        state_float = state_float / 255.
        
        # TODO return the predicted Q values for online/target function

## Act

The *act* function defines how agent reacts to current state(observation of environment). Given a state, the agent chooses the optimal action to perform based on the policy(Q function), or sometimes it would act randomly regardless of the policy to explore the state space. Please refer to the optimal action section in the cheatsheet. To choose an action, we need to predict the Q values for all possible actions in action dimension, and choose the one that gives the highest Q-value.
     

###    Small examples on NumPy indexing:

In [0]:
import numpy as np
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
a

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

To get element at 1st row, 2nd col:

In [0]:
b = a[0,1]
b

2

To get elements at (1st row, 2nd col) and (3rd row, 1st col):

In [0]:
c = a[[0,2],[1,0]]
c

array([2, 7])

To get max numbers in a along 1st axis:

In [0]:
d = np.max(a, axis = 0)
d

array([7, 8, 9])

To get the indices of max numbers in a along 1st axis:

In [0]:
e = np.argmax(a, axis=0);
e

array([2, 2, 2])

In [0]:
class DQNAgent:
    # def predict()
    
    def act(self, state):
        """Given a state, choose an epsilon-greedy action and update value of step
        Input
          state(np.array) 
            A single observation of the current state, dimension is (state_dim)
        Output
          action
            An integer representing which action agent will perform
        """
        # TODO choose action with epsilon-greedy policy
        if np.random.rand() < self.eps:
          # random action
          pass
        else:
          # policy action
          pass
          
        # decrease eps
        self.eps *= self.eps_decay
        self.eps = max(self.eps_min, self.eps)
        # increment step
        self.step += 1
        return action

## Remember

As described above, the agent needs to remember its past experiences. We declared a deque named *memory* that stores prior experiences. The agent uses experiences to learn how to update its value prediction in the future. Each time the agent performs an action, it collects an experience, please append the experience to *memory*

In [0]:
class DQNAgent:
    def remember(self, experience):
        """Add the experience to memory (deque)
        Input
          experience =  (state, next_state, action, reward, done) tuple
            Each time agent performs an action, it collects an experience which characterize the current state,
            action it performs under current state, the next state after performing the action, the reward it
            collects after performing the action, and whether the game is finished or not
        Output
            None
        """
        # TODO Add the experience to memory

## Learn




The learning process relies on the Q-learning algorithm in RL Cheatsheet. Specifically, we make the observation that the prediction based on reward and next action-state value is more accurate than predicting directly the current action-state pair. 

There are some key steps to perform:
- experience sampling: We will sample experience from all past experiences as the “training set” of the current iteration of improving the q value.


- predicting online q values: Using the sampled experience, we try to predict online q values (q values for current state)  for all state-action pairs in the sampled batch, by invoking the predict() method


- predicting target q values:
Using the sampled experience, we try to predict target q values (q values for next_state) for all state-action pairs in the sampled batch, using next_state and reward, also by invoking the predict() method


- calculate loss:
Similar to all machine learning models, we need to measure how our prediction(pred) fit the actual results(label), and improve our model by trying to minimize that difference between them. In this step, we calculate the loss(Huber loss between the predicted online q value(pred) and the predicted target q value(label)
- update online q function: Using the loss we computed, we update our online q function. That is how we keep improving the quality of our q value estimations by Temporal Difference Learning

In [0]:
class DQNAgent:  
    def learn(self):
        """Update prediction action value (Q) function with a batch of experiences
        """
        # set up and check learning criterion 

        # sample a batch of experiences from self.memory
        state, next_state, action, reward, done = sample_batch(self.memory, self.batch_size)

        # calculate prediction Q values for the batch
        pred_q = calculate_prediction_q(state, action)

        # calculate target Q values for the batch
        target_q = calculate_target_q(next_state, reward)

        # calculate huber loss of target and prediction values
        loss = calculate_huber_loss(pred_q, target_q)
        
        # update target network
        update_prediction_q(loss, optimizer)


### Experience Sampling 
Mario learns by drawing past experiences from its memory. The memory is a queue data structure that stores each individual experience in the format of 

> state, next_state, action, reward, done

Examples of some experiences in Mario's memory:


- state: ![pic](https://drive.google.com/uc?id=1D34QpsmJSwHrdzszROt405ZwNY9LkTej)  next_state: ![pic](https://drive.google.com/uc?id=13j2TzRd1SGmFru9KJImZsY9DMCdqcr_J) action: jump reward: 0.0 done: False


- state: ![pic](https://drive.google.com/uc?id=1ByUKXf967Z6C9FBVtsn_QRnJTr9w-18v) next_state: ![pic](https://drive.google.com/uc?id=1hmGGVO1cS7N7kdcM99-K3Y2sxrAFd0Oh) action: right reward: -10.0 done: True


- state: ![pic](https://drive.google.com/uc?id=10MHERSI6lap79VcZfHtIzCS9qT45ksk-) next_state: ![pic](https://drive.google.com/uc?id=1VFNOwQHGAf9pH_56_w0uRO4WUJTIXG90) action: right reward: -10.0 done: True


- state: ![pic](https://drive.google.com/uc?id=1T6CAIMzNxeZlBTUdz3sB8t_GhDFbNdUO) next_state: ![pic](https://drive.google.com/uc?id=1aZlA0EnspQdcSQcVxuVmaqPW_7jT3lfW) action: jump_right reward: 0.0 done: False


- state: ![pic](https://drive.google.com/uc?id=1bPRnGRx2c1HJ_0y_EEOFL5GOG8sUBdIo) next_state: ![pic](https://drive.google.com/uc?id=1qtR4qCURBq57UCrmObM6A5-CH26NYaHv) action: right reward: 10.0 done: False

State and next_state are observations at timestep *t* and *t+1* respectively. They are both of type `LazyFrame`, which allows us to optimize memory usage. To convert a `LazyFrame` to numpy array, do

```
state_np_array = np.array(state_lazy_frame)
```

Action represents what Mario takes when the state transition happens. 

Reward is the feedback from environment after transition happens. 

Done indicates if next_state is a terminal state, which means Mario is dead. Terminal state by definition has a return value of 0.


One question one might ask why do we want to sample data points from all past experiences rather than the most recent ones(for example, from the previous episode), which are newly trained with higher accuracy. 

The intuition is behind the tradeoff between these two approaches:

Do we want to train on data that are generated from a small-size dataset with relatively high quality or a huge-size dataset with relatively lower quality? 

The answer is the latter, because the more data we have, the more of a wholistic, comprehensive point of view we have on the overall behavior of the system we have, in our case, the Mario game. Limited size dataset has the danger of overfitting and overlooking bigger pictures of the entire action/state space. 


Remember, Reinforcement Learning is all about exploring different scenarios(state) and keeping improving based on trial and errors, generated from the interactions between the agent(action) and the environmental feedback(reward). 

## Instruction

Return a batch of experiences grouped by (state, next_state, action, reward, done) individually. Standardize all formats to numpy array. 

 

In [0]:
class DQNAgent:
  def sample_batch(memory, batch_size):
    """
    Input
      memory (FIFO queue)
        a queue where each entry has five elements as below
        state: LazyFrame of dimension (state_dim)
        next_state: LazyFrame of dimension (state_dim)
        action: integer, representing the action taken
        reward: float, the reward from state to next_state with action
        done: boolean, whether state is a terminal state
      batch_size (int)
        size of the batch to return 

    Output
      state, next_state, action, reward, done (tuple)
        a tuple of five elements: state, next_state, action, reward, done
        state: numpy array of dimension (batch_size x state_dim)
        next_state: numpy array of dimension (batch_size x state_dim)
        action: numpy array of dimension (batch_size)
        reward: numpy array of dimension (batch_size)
        done: numpy array of dimension (batch_size)
    """
    return (None, None, None, None, None)

### Predicted Q Value

The learning process relies on the Q-learning algorithm (refer to Q-learning in cheatsheet):

> Q_p(s, a) <- Q_p(s, a) + α(r + γ max Q_t(s', a') - Q_p(s,a))

where Q_p is the prediction value function, Q_t is the target value function, s and a are the current state and action, s' is the next state, a' is the best next action decided by Q_p and s' collectively. We use two separate neural networks to represent Q_p and Q_t. The neural networks learn to estimate state-action value (Q value) better over the learning process. All s, a and s' are retrieved from memory. 

The reason to have 2 value functions is to prevent divergence during the optimization. Q_p is used to make actual prediction of the current state-action value, while Q_t is used in conjunction with r to determine the target state-action value (refer to Temporal Difference Learning in Cheatsheet). In this section we make value prediction using Q_p. 

Ideally we pass both s and a to the Q_p function, which outputs the predicted value for the state-action pair. Imagine having 5 possible actions, this means passing the state-action pair to the Q_p neural network 5 times, which is very costly. To improve efficiency, we pass only the state to Q_p, which outputs predicted Q values for all possible actions at once. For example:

Input

state (s): ![pic](https://drive.google.com/uc?id=1ByUKXf967Z6C9FBVtsn_QRnJTr9w-18v)

Output
- moving right (a_1): -10
- jumping up (a_2): 10
- jumping right (a_3): 0

This gives us 

```
Q_p(s, a_1) = -10
Q_p(s, a_2) = 10
Q_p(s, a_3) = 0
```

In our scenario, since the action is given (e.g. a_2), we can directly return the associated Q value, i.e. Q_p(s, a_2). 

## Instruction

For a batch of experiences consisting of states (s) and actions (a), calculate the estimated value for each state-action pair Q(s, a). Return the results in `torch.tensor` format. 


In [0]:
class DQNAgent:
  # def predict()
  
  def calculate_prediction_q(state, action):
    """
    Input
      state (np.array)
        dimension is (batch_size x state_dim), each item is an observation 
        for the current state 
      action (np.array)
        dimension is (batch_size), each item is an integer representing the 
        action taken for current state 

    Output
      pred_q (torch.tensor)
        dimension of (batch_size), each item is a predicted Q value of the 
        current state-action pair 
    """
    return None

### Target Q Value

In this section we calculate the target Q value, in the form of

> r + γ max Q_t(s', a')

where r is the reward at transition from s to s',  γ  is the discounting factor, and s' is the next state. Because a' is not part of the actual experience (it is the predicted best action to take at next state), we will estimate it using our prediction value function Q_p by taking the argmax of Q_p(s', a') with respect to a'. 

> a' = argmax_a Q_p(s', a)

Target Q value, in comparison to prediction Q value Q_p(s, a), gives a better estimate of the current state-action value. We want to update the predicted Q value Q_p(s, a) towards target Q value, r + γ max Q_t(s', a'). 

In this section we calculate the target Q value of current state-action. 

## Instruction

For a batch of experiences consisting of next_states (s') and rewards (r), calculate the target Q value using above mentioned equation. Note that a' is not explicitly given, so we will need to first obtain that using prediction value function Q_p. 

Return the results in `torch.tensor` format. 

In [0]:
class DQNAgent:
  # def predict()
  
  def calculate_target_q(next_state, reward):
    """
    Input
      next_state (np.array)
        dimension is (batch_size x state_dim), each item is an observation 
        for the next state 
      reward (np.array)
        dimension is (batch_size), each item is a float representing the 
        reward collected from (state -> next state) transition 

    Output
      target_q (torch.tensor)
        dimension of (batch_size), each item is a target Q value of the current
        state-action pair, calculated based on reward collected and 
        estimated Q value for next state
    """
    return None

### Loss between Prediction and Target Q Value

To improve our value estimation, we would like our predicted Q value to be as close to the target Q value as possible. In other words, we want to minimize the distance between Q_p(s, a) and r + γ max Q_t(s', a'). To do this, we calculate the *huber loss* between the two values, and use this loss to update Q_p, the prediction value function. 


![pic](https://drive.google.com/uc?id=1FZM7sBnMgY5GQNTx-o3LtLRLQQM0mwat)


Huber loss is a smoothed version of L1 loss. Graph above gives some intuition behind L1 vs. L2 vs. Huber loss. L1 is intolerant around the origin and gives a high loss when there is only small difference between predicted and target value. On the other hand, L2 loss explodes quickly when there is a big difference between predicted and target value. Huber loss conveniently avoids both issues. 

Hint: the huber loss can be called in this way
```
loss = nn.functional.smooth_l1_loss(input, target)
```

## Instruction

Given predicted and target Q values for the batch of experiences, return the sum of huber loss. 


In [0]:
class DQNAgent:
  def calculate_huber_loss(pred_q, target_q):
    """
    Input
      pred_q (torch.tensor)
        dimension is (batch_size), each item is an observation 
        for the next state 
      target_q (torch.tensor)
        dimension is (batch_size), each item is a float representing the 
        reward collected from (state -> next state) transition 

    Output
      loss (torch.tensor)
        a single value representing the Huber loss of pred_q and target_q
    """
    return None

### Update Online_Q based on loss

After we calculated the loss between pred_q and target_q, we need to update the policy to reduce the decrepancy between them. To update the policy (the parameters in nueral network), we need to use optimizer which backpropages the loss to parameters in neural network

In [0]:
class DQNAgent:
  def __init__(self, ...):
    # optimizer updates parameters in online_q using backpropagation
    self.optimizer = torch.optim.Adam(self.online_q.parameters(), lr=0.00025)

  def update_prediction_q(self, loss, optimizer):
    '''
    Input
      loss (torch.tensor)
        a single value representing the Huber loss of pred_q and target_q
      optimizer
        optimizer updates parameter in our online_q neural network to reduce
        the loss
    '''
    pass
    



With all the helper methods implemented, let's revisit our learn function. 

We need to set up the learning process and check some criterion. Logic is added for you. 

In [0]:
class DQNAgent:
    def learn(self):
        """Update prediction action value (Q) function with a batch of experiences
        """
        # sync target network
        if self.step % self.copy_every == 0:
            self.sync_target_q()
        # checkpoint model
        if self.step % self.save_every == 0:
            self.save_model()
        # break if burn-in
        if self.step < self.burnin:
            return
        # break if no training
        if self.step % self.learn_every != 0:
            return

        # sample a batch of experiences from self.memory
        state, next_state, action, reward, done = sample_batch(self.memory, self.batch_size)

        # calculate prediction Q values for the batch
        pred_q = calculate_prediction_q(state, action)

        # calculate target Q values for the batch
        target_q = calculate_target_q(next_state, reward)

        # calculate huber loss of target and prediction values
        loss = calculate_huber_loss(pred_q, target_q)
        
        # update target network
        update_prediction_q(loss, optimizer)


## Save

We know have all the key functionlities realized. There are some additional helper methods that would be useful. For example:
- to be able to save the agent

We implemented these methods for you, and here is your completed agent file. 

In [0]:
from collections import deque
import torch
import torch.nn as nn
import numpy as np
import random
from neural import ConvNet
import pdb

class DQNAgent:
    def __init__(self, state_dim, action_dim, max_memory, double_q):
        # state space dimension
        self.state_dim = state_dim
        # action space dimension
        self.action_dim = action_dim
        # replay buffer
        self.memory = deque(maxlen=max_memory)
        # if double_q, use best action from online_q for next state q value
        self.double_q = double_q
        # future reward discount rate
        self.gamma = 0.9
        # initial epsilon(random exploration rate)
        self.eps = 1
        # final epsilon
        self.eps_min = 0.1
        # epsilon decay rate
        self.eps_decay = 0.99999975
        # current step, updated everytime the agent acts
        self.step = 0
        # number of experiences between updating online q
        self.learn_every = 3
        # number of experiences to collect before training
        # self.burnin = 1e5
        self.burnin = 1e2
        # number of experiences between updating target q with online q
        self.copy_every = 1e4
        # number of experiences between saving the current agent
        self.save_every = 5e5

        # batch size used to update online q
        self.batch_size = 32
        # online action value function, Q(s, a)
        self.online_q = ConvNet(input_dim=state_dim, output_dim=action_dim)
        # target action value function, Q'(s, a)
        self.target_q = ConvNet(input_dim=state_dim, output_dim=action_dim)
        # optimizer
        self.optimizer = torch.optim.Adam(self.online_q.parameters(), lr=0.00025)

    def predict(self, state, model):
        """Given a state, predict Q values of all actions
        model is either 'online' or 'target'
        """
        state_float = torch.tensor(np.array(state)).float() / 255.
        if model == 'online':
            return self.online_q(state_float)
        if model == 'target':
            return self.target_q(state_float)

    def act(self, state):
        """Given a state, choose an epsilon-greedy action
        """
        if np.random.rand() < self.eps:
            # random action
            action = np.random.randint(low=0, high=self.action_dim)
        else:
            # policy action
            q = self.predict(np.expand_dims(state, 0), model='online')
            action = torch.max(q, axis=1)[1].item()
        # decrease eps
        self.eps *= self.eps_decay
        self.eps = max(self.eps_min, self.eps)
        # increment step
        self.step += 1
        return action

    def remember(self, experience):
        """Add the observation to memory
        """
        self.memory.append(experience)

    def learn(self):
        """Update online action value (Q) function with a batch of experiences
        """
        # sync target network
        if self.step % self.copy_every == 0:
            self.sync_target_q()
        # checkpoint model
        if self.step % self.save_every == 0:
            self.save_model()
        # break if burn-in
        if self.step < self.burnin:
            return
        # break if no training
        if self.step % self.learn_every != 0:
            return
        # sample batch
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(np.array, zip(*batch))
        # get next q values from target_q
        next_q = self.predict(next_state, 'target')
        # calculate discounted future reward
        if self.double_q:
            q = self.predict(next_state, 'online')
            q_idx = torch.max(q, axis=1)[1]
            target_q = torch.tensor(reward) + torch.tensor(1. - done) * self.gamma * next_q[np.arange(0, self.batch_size), q_idx]
        else:
            target_q = torch.tensor(reward) + torch.tensor(1. - done) * self.gamma * torch.max(next_q, axis=1)[0]
        # get predicted q values from online_q and actions taken
        curr_q = self.predict(state, 'online')
        pred_q = curr_q[np.arange(0, self.batch_size), action]
        # huber loss
        loss = nn.functional.mse_loss(pred_q, target_q)
        # update online_q
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def save_model(self):
        """Save the current agent
        """
        return

    def sync_target_q(self):
        """Update target action value (Q) function with online action value (Q) function
        """
        self.target_q.load_state_dict(self.online_q.state_dict())

# Start Learning! 

After finishing the implementing the Reinforcement Learning model and the environment wrappers, we are ready to build our main driver function that will get our Mario up and running. The purpose of the `main` function is to call the agent and environment wrappers we already built.

We will a big `while` loop that will keep running until we quit game or stop training(by checking the `done` variable), and repeat again for `num_episodes` number of episodes.

The meat of the algorithm is in the main loop, let's take a closer look: 



**Instruction:**

1.At the beginning of a new episode, we need to reinitialize the `state` by calling `env.reset()`

2.Then we need several variables to hold the logging information we collected in this episode:

`ep_reward`: reward collected in this episode

`ep_num_steps`: total number of actions perforemed in this episode
    
`ep_total_loss`: total loss collected in this episode

`ep_total_q`: total loss collected in this episode

`ep_learn_length`: used for mean loss/q_value


3.Now we are inside the while loop that plays the game, and we can call `env.render()` to display the visual

4.We want to apply the action policy on the current state by calling `agent.act`, remember action policy  $pi$ characterizes how the agent reacts to environment.

5.Then after the agent performs the action, the environment will ouput its feedback, including information such as next state, reward, if Mario is dead(`done`), and some other info. Calling `env.step` with the agent's action should do the trick.

6.Agent needs to remember the experience in this action he takes in this state, call `agent.remember` using all the environment output from the step above.

7.Call `agent.learn`

8.Update logging info

9.Update state to get the evironment ready for the next interation of agent-environment/action-reward interactions


In [0]:
### for Loop that train the model num_episodes times by playing the game

for e in range(num_episodes):

    # 1. Reset env/restart the game
    state = None

    # 2. Logging
    ep_reward = 0.0
    ep_length = 0
    ep_total_loss = 0.0
    ep_total_q = 0.0
    ep_learn_length = 1 # used for mean loss/q_value

    # Play the game!
    while True:

        # 3. Show environment (the visual)

        # 4. Run agent on the state
        action = None

        # 5. Agent performs action
        next_state, reward, done, info = None

        # 6. Remember
        agent.remember(experience=(None,None,None,None))

        # 7.Learn (conditional) (80% time cost)
        q_value, loss = None

        # 8.Logging
        ep_reward += reward
        ep_length += 1
        if q_value and loss:
            ep_total_loss += loss
            ep_total_q += q_value
            ep_learn_length += 1

        # Update state
        state = None

        # If done break loop
        if done or info['flag_get']:
            break



Below is the fully functional `main` class, we added logging info that will help keep track of the status of the training. 

We have helped you initialize and applied the wrappers for the environment for you, and we also initialized the agent.

In addition, we also added model saving functionality for you so that you can replay the model you trained.

In [0]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
import numpy as np
import pdb
import time
# original environment
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')

# define action space on the environment:
# NOOP: no action
# right: walk right
# right, A: jump right
# right, B: run right
env = JoypadSpace(
    env,
    [['right'],
    ['right', 'A']]
    )

## apply environment wrappers
env = apply_wrappers_to_env(env)

# After applying environment wrappers, observation space (a.k.a state_dim) shrinks from
# 240 (height) x 256 (width) x 3 (RGB color channels) 
# to 
# 4 (#frames) x 84 (height) x 84 (width)

# dimensional parameters after reshaping
state_dim = (4,84,84)
action_dim = env.action_space.n

# Intialize agent
agent = DQNAgent(state_dim=state_dim, action_dim=action_dim, max_memory=100000, double_q=True)

# Logs
log = {
    "rewards": [],
    "lengths": [],
    "losses": [],
    "q_values": []
}
log_file = os.path.join(agent.save_dir, "log.txt")

# Timing
start = time.time()
step = 0

# number of episodes
num_episodes = 10000

# Main training loop
for e in range(num_episodes):

    # 1. Reset env/restart the game
    state = None

    # 2. Logging
    ep_reward = 0.0
    ep_length = 0
    ep_total_loss = 0.0
    ep_total_q = 0.0
    ep_learn_length = 1 # used for mean loss/q_value

    # Play the game!
    while True:

        # 3. Show environment (the visual)

        # 4. Run agent on the state
        action = None

        # 5. Agent performs action
        next_state, reward, done, info = None

        # 6. Remember
        agent.remember(experience=(None,None,None,None))

        # 7.Learn (conditional) (80% time cost)
        q_value, loss = None

        # 8.Logging
        ep_reward += reward
        ep_length += 1
        if q_value and loss:
            ep_total_loss += loss
            ep_total_q += q_value
            ep_learn_length += 1

        # Update state
        state = None

        # If done break loop
        if done or info['flag_get']:
            break

    # Log info in this episode
    log["rewards"].append(ep_reward)
    log["lengths"].append(ep_length)
    log["losses"].append(np.round(ep_total_loss/ep_learn_length, 5))
    log["q_values"].append(np.round(ep_total_q/ep_learn_length, 5))

    # Print & Log every 50th episode
    if e % 50 == 0:
        mean_reward = np.round(np.mean(log['rewards'][-100:]), 3)
        mean_length = np.round(np.mean(log['lengths'][-100:]), 3)
        mean_loss = np.round(np.mean(log['losses'][-100:]), 3)
        mean_q_value = np.round(np.mean(log['q_values'][-100:]), 3)
        eps = np.round(agent.eps, 3)
        step_time = np.round((time.time() - start_time)/(agent.step - start_step), 3)
        start_time = time.time()
        start_step = agent.step
        print(
            f"Episode {e} - "
            f"Step {agent.step} - "
            f"Step Time {step_time} - "
            f"Epsilon {eps} - "
            f"Mean Reward {mean_reward} - "
            f"Mean Length {mean_length} - "
            f"Mean Loss {mean_loss} - "
            f"Mean Q Value {mean_q_value} - "
            f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        )

        with open(log_file, "a") as f:
            f.write(
                f"{e:8d}{agent.step:10d}{eps:10.3f}"
                f"{mean_reward:15.3f}{mean_length:15.3f}{mean_loss:15.3f}{mean_q_value:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )

        # Running on Colab, download checkpoints to local
        if 'google.colab' in sys.modules:
            from google.colab import files
            files.download(os.path.join(agent.save_dir, "online_q_1.chkpt"))